# 爬取静态网页的一个实例

## 导入requests包和BeautifulSoup模块

In [1]:
import requests
from bs4 import BeautifulSoup

url = "http://www.52xlzx.com/guestbook/?fid=0&page=70"

headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36',
}
resp = requests.get(url, headers=headers)

In [2]:
resp.status_code

200

## 测试单个页面的爬取结果，处理出现的错误

In [3]:
resp.encoding = 'gbk'
soup = BeautifulSoup(resp.text, 'html.parser')
liuyan_list = soup.find_all('table', {'class':'guestbook'})
print(len(liuyan_list))
string1 = liuyan_list[9].find('td',{'class':'content'}).text.split('留言回复')[0].strip()
print(string1)
print(len(liuyan_list[9].find('td',{'class':'content'}).text.split('留言回复')))
#string2 = liuyan_list[9].find('td',{'class':'content'}).text.split('留言回复')[1].split("(署名")[0].strip()
#string3 = "署名" + liuyan_list[0].find('td',{'class':'content'}).text.split('留言回复')[1].split("(署名")[1][:-1].strip()

20
有得抑郁症的人为什么早上不起床
1


## 写循环爬取所有页面

In [4]:
import pandas as pd
url_list = []
data_list1 = []
data_list2 = []
data_list3 = []

for i in range(1,272):
    url = "http://www.52xlzx.com/guestbook/?fid=0&page={}".format(i)
    url_list.append(url)

def getInfo(urlList):
    n = 1
    for url in urlList:
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')
        liuyan_list = soup.find_all('table', {'class':'guestbook'})
        m = 1
        for liuyan in liuyan_list:
            data1 = liuyan.find('td',{'class':'content'}).text.split('留言回复')[0].strip()
            if len(liuyan.find('td',{'class':'content'}).text.split('留言回复')) == 1:
                data2 = ""
                data3 = ""
            else:
                data2 = liuyan.find('td',{'class':'content'}).text.split('留言回复')[1].split("(署名")[0].strip()
                data3 = "署名" + liuyan.find('td',{'class':'content'}).text.split('留言回复')[1].split("(署名")[1][:-1].strip()
            data_list1.append(data3)
            data_list2.append(data1)
            data_list3.append(data2)
            #print("第{}页第{}条已抓取成功".format(n,m))
            m = m + 1
        n = n + 1

In [ ]:
getInfo(url_list)

In [ ]:
data = {"署名/日期":data_list1, "问题":data_list2, "回复":data_list3}
df = pd.DataFrame(data, columns=["署名/日期", "问题", "回复"])
df.head()

In [ ]:
df.shape

## 写入到本地

In [ ]:
df.to_csv('liuyan.csv', sep='\t', index=False)